In [2]:
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
sns.set()

### Set the data for the tests

In [3]:
# read Ospiti.csv
df = pd.read_csv('input/Ospiti.csv')
df["Data"] = pd.to_datetime(df["Data"]).dt.date
df.set_index("Data",inplace=True)
df = df.iloc[:-1]
df

,Ospiti
Data,
2021-06-01,319.0
2021-06-02,396.0
2021-06-03,188.0
2021-06-04,258.0
2021-06-05,571.0
...,...
2022-11-25,375.0
2022-11-26,732.0
2022-11-27,495.0


In [4]:
# divide the dataframe incrementally by one day (start with 365 days)
df_list = []
for i in range(0, len(df) - 364, 1):
    df_list.append(df.iloc[:i+365])
df_list


[            Ospiti
 Data              
 2021-06-01   319.0
 2021-06-02   396.0
 2021-06-03   188.0
 2021-06-04   258.0
 2021-06-05   571.0
 ...            ...
 2022-05-28   646.0
 2022-05-29   579.0
 2022-05-30   255.0
 2022-05-31   329.0
 2022-06-01   339.0
 
 [365 rows x 1 columns],
             Ospiti
 Data              
 2021-06-01   319.0
 2021-06-02   396.0
 2021-06-03   188.0
 2021-06-04   258.0
 2021-06-05   571.0
 ...            ...
 2022-05-29   579.0
 2022-05-30   255.0
 2022-05-31   329.0
 2022-06-01   339.0
 2022-06-02   557.0
 
 [366 rows x 1 columns],
             Ospiti
 Data              
 2021-06-01   319.0
 2021-06-02   396.0
 2021-06-03   188.0
 2021-06-04   258.0
 2021-06-05   571.0
 ...            ...
 2022-05-30   255.0
 2022-05-31   329.0
 2022-06-01   339.0
 2022-06-02   557.0
 2022-06-03   357.0
 
 [367 rows x 1 columns],
             Ospiti
 Data              
 2021-06-01   319.0
 2021-06-02   396.0
 2021-06-03   188.0
 2021-06-04   258.0
 2021-06-05   571.0

In [5]:
len(df_list)

182

### Get average predictions auto arima

In [6]:
import my_auto_arima

# create a dataframe with the predictions for each day
df_pred_autoarima = pd.DataFrame()
for i in range(len(df_list)):
    df_pred_autoarima = df_pred_autoarima.append(my_auto_arima.predictions(df_list[i].reset_index()));
    #df_pred_autoarima = pd.concat([df_pred_autoarima, my_auto_arima.predictions(df_list[i].reset_index())], axis=1);

/usr/local/lib/python3.8/dist-packages/statsforecast/arima.py:861: UserWarning: possible convergence problem: minimize gave code 2]
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/statsforecast/arima.py:861: UserWarning: possible convergence problem: minimize gave code 2]
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/statsforecast/arima.py:861: UserWarning: possible convergence problem: minimize gave code 2]
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/statsforecast/arima.py:861: UserWarning: possible convergence problem: minimize gave code 2]
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/statsforecast/arima.py:861: UserWarning: possible convergence problem: minimize gave code 2]
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/statsforecast/arima.py:861: UserWarning: possible convergence problem: minimize gave code 2]
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/statsforecast/arima.py:861: UserWarning: possible convergence probl

In [7]:
df_pred_autoarima = df_pred_autoarima.groupby(by=df_pred_autoarima.index).mean()
df_pred_autoarima

,AutoArima Forecasts
Data,
2022-06-02,397.950219
2022-06-03,459.580753
2022-06-04,634.782739
2022-06-05,523.540613
2022-06-06,217.089558
...,...
2022-12-02,409.874366
2022-12-03,760.022672
2022-12-04,485.478291


In [10]:
# save the predictions in a csv file
df_pred_autoarima.to_csv('output/predictions_autoarima_avg.csv')